Stitch the patched images in OCT_Patch/GSS_Patch folders back to the original images 
Expected outputs take the appearances of Cropped_OCT/Cropped_GSS

In [ ]:
# Create folders containing all patched images for each original image
import os
import numpy as np
import tifffile as tiff
import shutil

root_directory = 'CISL_Dataset/Train_Data/'
path1 = '/projectnb/cs585bp/kontact'
path2 = 'CISL_Dataset/Train_Data'
joined_path = os.path.join(path1, path2)

for path, subdirs, files in os.walk(joined_path):
    directory = path.split(os.path.sep)[-1]
    
    if directory == 'OCT_Patch':
        final_oct_dir = os.path.join(root_directory, 'Final_OCT')
        patch_files = sorted(os.listdir(path))
        orig_images = {}
        for patch_file in patch_files:
            if patch_file.endswith('.tif'):
                # Get the original filename: ex, crop_x_mus_1
                orig_filename = 'final_crop' + '_' + '_'.join(patch_file.split('_', 1)[1:]).split('.')[0]
                
                # Create a directory with the same name as the original filename
                output_dir = os.path.join(final_oct_dir, orig_filename)
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                
                # Copy the patch file to the output directory
                patch_path = os.path.join(path, patch_file)
                output_file = os.path.join(output_dir, patch_file)
                shutil.copy(patch_path, output_file)
                
    elif directory == 'GSS_Patch':
        final_gss_dir = os.path.join(root_directory, 'Final_GSS')
        patch_files = sorted(os.listdir(path))
        orig_images = {}
        for patch_file in patch_files:
            if patch_file.endswith('.tif'):
                # Get the original filename: ex, crop_y_gallyas_1
                orig_filename = 'final_crop' + '_' + '_'.join(patch_file.split('_', 1)[1:]).split('.')[0]
                
                # Create a directory with the same name as the original filename
                output_dir = os.path.join(final_gss_dir, orig_filename)
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                
                # Copy the patch file to the output directory
                patch_path = os.path.join(path, patch_file)
                output_file = os.path.join(output_dir, patch_file)
                shutil.copy(patch_path, output_file)
                

In [19]:
# Within each folders containing patched images associated with the original image, 
# patch them to recreate the original image
from PIL import Image
import os

path = '/projectnb/cs585bp/kontact/CISL_Dataset/Train_Data/Final_OCT/final_crop_x_mus_1'
os.chdir(path)
current_dir = os.getcwd()
print(current_dir)

# Manually set the width of the original image here:
# If 2560x1536 = WxH
max_width = 2560

# Get a list of all the image files in the folder
image_files = sorted([f for f in os.listdir(path) if f.endswith('.tif')])

# create an empty list to store the loaded images
images = []

# Load the images and append them to the list
for file in image_files:
    file_path = os.path.join(path, file)
    image = Image.open(file_path)
    images.append(image)

# Create a new image with the first image
new_image = images[0]

# Get the width of the first image
total_width = new_image.size[0]

# Keep track of the number of stitched images
num_images = 1

# Create a list to store the saved stitched images
saved_images = []

# Iterate over the remaining images and paste them onto the new image
for image in images[1:]:
    # Check if adding the next image would exceed the maximum width
    if total_width + image.size[0] > max_width:
        # Save the current stitched image
        new_image.save(f'stitched_image_{num_images}.jpg')
        saved_images.append(new_image)
        num_images += 1
        # Create a new image with the next image
        new_image = image
        total_width = new_image.size[0]
    else:
        # Note: image.size[0] = W, image.size[1] = H
        # Create a new image with the dimensions of the first image
        total_image = Image.new('L', (new_image.size[0] + image.size[0], new_image.size[1]))
        # Paste the previous stitched image onto the new image
        total_image.paste(new_image, (0, 0))
        # Paste the new image onto the previous stitched image
        total_image.paste(image, (new_image.size[0], 0))
        total_width += image.size[0]
        new_image = total_image

# Create a new vertically stacked image with all the saved stitched images
first_image = saved_images[0]
total_height = first_image.size[1] * len(saved_images)
new_image = Image.new('L', (first_image.size[0], total_height))
y_offset = 0
for image in saved_images:
    new_image.paste(image, (0, y_offset))
    y_offset += image.size[1]

# Save the final vertically stacked image
new_image.save('final_stitched_image.jpg')

/projectnb/cs585bp/kontact/CISL_Dataset/Train_Data/Final_OCT/final_crop_x_mus_1
